In [ ]:
import collections
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../resources/data/random_users_data_50.csv', encoding='utf-8')

In [ ]:
df

In [ ]:
# Crie um grafo direcionado no NetworkX
G = nx.DiGraph()

# Adicione nós para os usuários
for _, row in df.iterrows():
    G.add_node(row["Nome de usuário"])
    
    # Adicione as relações de seguindo (following)
    following_users = str(row["Following Users"]).split(", ")
    for following_user in following_users:
        if following_user.strip():  # Verifique se a string não está vazia
            G.add_edge(row["Nome de usuário"], following_user)
        
    # Adicione as relações de seguidores (followers)
    followers_users = str(row["Followers Users"]).split(", ")
    for follower_user in followers_users:
        if follower_user.strip():  # Verifique se a string não está vazia
            G.add_edge(follower_user, row["Nome de usuário"])

# Análise da Estrutura da rede

A rede analisada refere-se à rede de usuários do GitHub, uma plataforma de desenvolvimento colaborativo de software. Nesta rede, os "nós" representam os usuários individuais do GitHub, cada um com um perfil que inclui informações pessoais, repositórios de código-fonte, atividades, seguidores e usuários que estão seguindo. Os "arestas" na rede representam as conexões entre os usuários, que podem ser de dois tipos:

Seguindo (`Following`): Uma aresta entre o usuário A e o usuário B indica que o usuário A está seguindo o usuário B.

Seguidores (`Followers`): Uma aresta entre o usuário B e o usuário A indica que o usuário B está seguindo o usuário A.

Em resumo, os "nós" são os perfis de usuários individuais no GitHub, e as "arestas" representam as conexões de "seguindo" e "seguidores" entre esses usuários. A rede é usada para rastrear o relacionamento de seguidores e seguindo de usuários, bem como para facilitar a colaboração e a descoberta de projetos de código aberto e repositórios interessantes. A análise dessa rede pode fornecer insights sobre a interação e a influência dos usuários na plataforma do GitHub.

Abaixo Temos o números de nós e arestas do grafo

In [ ]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print(f'numero de nós {num_nodes}')
print(f'numero de arestas {num_edges}')


## Distribuição de nós

In [ ]:
in_degrees = dict(G.in_degree())
in_degree_sequence = list(in_degrees.values())

# Calcule a distribuição do grau
degree_counts = collections.Counter(in_degree_sequence)

# Ordene os graus e suas contagens
degree, count = zip(*degree_counts.items())

# Plote a distribuição do grau
plt.bar(degree, count)
plt.xlabel('Grau do Nó')
plt.ylabel('Número de Nós')
plt.title('Distribuição do Grau dos Nós')
plt.show()

In [ ]:
# d) Coeficiente de Clusterização
cluster_coefficients = nx.clustering(G)
average_cluster_coefficient = sum(cluster_coefficients.values()) / len(cluster_coefficients)
print(f'Coeficiente de Clusterização Global: {average_cluster_coefficient:.2f}')


In [ ]:
# Calcular o diâmetro em cada componente fortemente conectado
strongly_connected_components = list(nx.strongly_connected_components(G))
diameters = []
for component in strongly_connected_components:
    subgraph = G.subgraph(component)
    if subgraph.number_of_nodes() > 1:
        diameter = nx.diameter(subgraph)
        diameters.append(diameter)
    else:
        diameters.append(0)

print(f'Diâmetros dos componentes fortemente conectados: {diameters}')


In [ ]:
# Calcular os graus dos nodos
degrees = dict(G.degree())  # Dicionário com os graus dos nodos

# Plotar a distribuição de grau com nodos
plt.figure(figsize=(12, 6))
plt.hist(list(degrees.values()), bins=50, density=True, alpha=0.7, color='b')
plt.xlabel('Grau dos Nodos')
plt.ylabel('Densidade')
plt.title('Distribuição de Grau dos Nodos')
plt.show()

In [ ]:
# Calcule a distribuição de grau em escala log-log
log_degrees = np.log(degrees)
log_counts = np.log([degrees.count(degree) for degree in degrees])

# Ajuste uma reta (regressão linear) para os dados log-log
coefficients = np.polyfit(log_degrees, log_counts, 1)

# O coeficiente alfa é o coeficiente angular da reta de melhor ajuste
alfa = -coefficients[0]

print(f'Coeficiente alfa estimado: {alfa}')


In [ ]:
# Calcule o grau médio do grafo
average_degree = sum(degrees) / len(degrees)
print(f'Grau médio do grafo: {average_degree}')

In [ ]:
# Layout do grafo com layout fruchterman_reingold
pos = nx.spring_layout(G, k=0.2)

# Plote o grafo
plt.figure(figsize=(16, 16))

# Desenhe os rótulos dos nós
nx.draw_networkx_labels(G, pos, font_color='black', font_size=8)

# Ajuste o tamanho dos nós
node_size = 50

# Desenhe o grafo
nx.draw(G, pos=pos, node_size=node_size, with_labels=False, alpha=0.6, node_color='blue', edge_color='gray')

plt.title("Grafo de Relações do GitHub")
plt.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Generate Graph.gml to Gephi Software

# nx.write_gml(G, "../resources/gephi/graph.gml")